In [6]:
from dotenv  import load_dotenv
import boto3
import os
import pandas as pd

load_dotenv("../.env")
aws_access_key_id: str = os.environ.get("AWS_ACCESS_KEY_ID")
aws_secret_access_key: str = os.environ.get("AWS_SECRET_ACCESS_KEY")
aws_bucket_name: str = os.environ.get("AWS_BUCKET_NAME")

prefix: str = "ajio-myntra"

# Crea una instancia del cliente de S3
s3 = boto3.client(
    's3',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
)

In [ ]:
# list all buckets
buckets = [bucket["Name"] for bucket in s3.list_buckets()["Buckets"]]

In [ ]:
# list all objects in a bucket 
response = s3.list_objects_v2(Bucket=aws_bucket_name, Prefix=prefix)

if 'Contents' in response:
    objects = response['Contents']
    for obj in objects:
        print(obj['Key'])
else:
    print('No se encontraron objetos en el bucket.')

In [ ]:
# download json
path_json_s3: str = "ajio-myntra/alternative/20231213/Myntra__Marianfer_Cruz_20231213_143830.success.json"
local_json_path: str = "../data/json/Myntra__Marianfer_Cruz_20231213_143830.success.json"

if not os.path.exists(local_json_path):

    s3.download_file(
        aws_bucket_name, 
        path_json_s3,
        local_json_path
        )
else:
    print("El archivo ya existe.")

In [ ]:
# donwload images
path_img_s3: str = "ajio-myntra/alternative/20231213/j_lq3uhp5g14myh50mul.fda76d907cf88f51174e77caf77803459ef3174f.file_lq3v0p90103owhbiyl.[_].jpg"
local_img_path: str = "../data/images/test2.jpg"

if not os.path.exists(local_img_path):

    s3.download_file(
        aws_bucket_name, 
        path_img_s3,
        local_img_path
        )
else:
    print("El archivo ya existe")

In [23]:
# validar rutas 
route = "ajio-myntra/origin/20231214/New_collector_20231214_154733.success.json"
response = s3.list_objects_v2(Bucket=aws_bucket_name, Prefix=route)

if "Contents" in response:
    print(True)
else:
    print(False)

False


In [21]:
# validar objetos 
path = "/ajio-myntra/origin/20231214/New_collector_20231214_154733.success.json", "ajio-myntra/alternative/Myntra__Marianfer_Cruz_20231213_130659.json"
s3.head_object(Bucket=aws_bucket_name, Key=path[1])

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found